# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


## 0. Initiate 

In [27]:
import pandas as pd
import numpy as np
import math
import json
from sklearn.preprocessing import MultiLabelBinarizer 
%matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

In [28]:
transcript2 = pd.read_json('data/transcript.json', orient='records', lines=True)

In [29]:
transcript2.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


## 1. Clean and Organize Data

In [30]:
def clean_and_organize_data(portfolio=portfolio,profile=profile,transcript=transcript):
    ## Portfolio
    portfolio.rename(columns={'id':'offer_id'},inplace=True)
    mlb = MultiLabelBinarizer()
    portfolio = portfolio.join(pd.DataFrame(mlb.fit_transform(portfolio.pop('channels')),
                          columns=mlb.classes_,
                          index=portfolio.index))
    portfolio = portfolio.merge(pd.get_dummies(portfolio['offer_type'],prefix='offer',prefix_sep='_x_'),left_index=True,right_index=True)
    portfolio.drop(columns='offer_type',inplace=True)
    portfolio.set_index('offer_id')
    ## Profile
    profile = profile.dropna(axis = 0)
    profile.loc[:,'became_member_on']=pd.to_datetime(profile['became_member_on'],format='%Y%m%d')
    profile = profile.merge(pd.get_dummies(profile['gender'],prefix='gender',prefix_sep='_x_'),left_index=True,right_index=True)
    profile = profile.drop(columns='gender')
    profile.set_index('id',inplace=True)
    ## Transcript
    transcript = transcript.merge(transcript.value.apply(pd.Series),left_index=True,right_index=True)
    transcript.loc[transcript['offer_id'].isna(),'offer_id'] = transcript[transcript['offer_id'].isna()]['offer id']
    transcript = transcript.merge(pd.get_dummies(transcript['event'],prefix='event',prefix_sep='_x_'),left_index=True, right_index=True)
    transcript.drop(columns = ['offer id','value'],inplace=True)
    col_list = ['time','person',
                 'offer_id',
                 'amount',
                 'reward',
                 'event_x_offer received',
                 'event_x_offer viewed',
                 'event_x_transaction',
               'event_x_offer completed']
    transcript = transcript[col_list]
    transcript = transcript[transcript['person'].isin(profile.index)].reset_index()
    
    return portfolio,profile,transcript

In [31]:
portfolio_new,profile_new,transcript_new = clean_and_organize_data()

C:\Users\thoma\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [32]:
portfolio.head() 

,difficulty,duration,offer_id,offer_type,reward
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [33]:
portfolio.shape

(10, 5)

In [34]:
profile.head() 

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [35]:
profile.shape 

(17000, 5)

In [36]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [37]:
transcript.shape

(306534, 4)

In [ ]:
transcript = transcript.merge(transcript.value.apply(pd.Series),left_index=True,right_index=True)
transcript.loc[transcript['offer_id'].isna(),'offer_id'] = transcript[transcript['offer_id'].isna()]['offer id']
transcript = transcript.merge(pd.get_dummies(transcript['event'],prefix='event',prefix_sep='_x_'),left_index=True, right_index=True)
transcript.drop(columns = ['offer id','value','event'],inplace=True)
col_list = ['time','person',
             'offer_id',
             'amount',
             'reward',
             'event_x_offer received',
             'event_x_offer viewed',
             'event_x_transaction',
           'event_x_offer completed']
transcript = transcript[col_list]
transcript = transcript[transcript['person'].isin(profile.index)].reset_index()

In [21]:
offer = transcript[transcript['event']=='offer received']
offer.head(3)

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}


In [18]:
transcript.event.unique()

array(['offer received', 'offer viewed', 'transaction', 'offer completed'],
      dtype=object)

In [22]:
viewed = transcript[transcript['event']=='offer viewed']
viewed.head(3)

,event,person,time,value
12650,offer viewed,389bc3fa690240e798340f5a15918d5c,0,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'}
12651,offer viewed,d1ede868e29245ea91818a903fec04c6,0,{'offer id': '5a8bc65990b245e5a138643cd4eb9837'}
12652,offer viewed,102e9454054946fda62242d2e176fdce,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


## 2. Explore Data

In [51]:
tot_offers = transcript['event_x_offer received'].sum()
tot_viewed = transcript['event_x_offer viewed'].sum()
tot_completed = transcript['event_x_offer completed'].sum()
tot_num_purchases = transcript['event_x_transaction'].sum()
tot_sum_purchases = transcript['amount'].sum()
tot_sum_reward = transcript['reward'].sum()

print('Percentage viewed : {0:.2f}%'.format(100* tot_viewed/tot_offers))
print('Percentage completed : {0:.2f}%'.format(100*tot_completed/tot_offers))
print('Percentage of purchases that were rewarded  : {0:.2f}%'.format(100*tot_completed/tot_num_purchases))

Percentage viewed : 74.98%
Percentage completed : 48.79%
Percentage of purchases that were rewarded  : 26.17%


In [53]:
transcript['purc_rewarded'] = np.nan

transcript['purc_viewed'] = False

In [107]:
test_1_ids =['e2127556f4f64592b11af22de27a7932'] #'c'

In [108]:
test_1 = transcript[transcript['person'].isin(test_1_ids)]

In [121]:
test_1

,index,time,person,offer_id,amount,reward,event_x_offer received,event_x_offer viewed,event_x_transaction,event_x_offer completed,purc_rewarded,purc_viewed
1,2,0,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,1,0,0,0,NaN,False
17760,20283,18,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,0,1,0,0,NaN,False
91402,102635,288,e2127556f4f64592b11af22de27a7932,NaN,17.88,NaN,0,0,1,0,NaN,False
92480,103834,294,e2127556f4f64592b11af22de27a7932,NaN,21.43,NaN,0,0,1,0,NaN,False
98717,110830,336,e2127556f4f64592b11af22de27a7932,3f207df678b143eea3cee63160fa8bed,NaN,NaN,1,0,0,0,NaN,False
133962,150600,408,e2127556f4f64592b11af22de27a7932,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,1,0,0,0,NaN,False
152024,171209,420,e2127556f4f64592b11af22de27a7932,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,0,1,0,0,NaN,False
179364,201574,504,e2127556f4f64592b11af22de27a7932,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,1,0,0,0,NaN,False
199950,225059,522,e2127556f4f64592b11af22de27a7932,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,0,1,0,0,NaN,False
199951,225060,522,e2127556f4f64592b11af22de27a7932,NaN,18.42,NaN,0,0,1,0,NaN,False


In [ ]:
users = profile.index.tolist()
for usr in users:
    viewed = transcript[(transcript['person']==usr)&(transcript['event_x_offer viewed']==1)][['offer_id']]
    offered = transcript[(transcript['person']==usr)&(transcript['event_x_offer received']==1)][['offer_id']]
    offer_viewed = np.intersect1d(offered.values,viewed.values).tolist()
    transcript.loc[(transcript['offer_id'].isin(offer_viewed))&(transcript['person']==usr),'purc_viewed']=True

In [110]:
viewed = test_1[test_1['event_x_offer viewed']==1][['offer_id']]

In [110]:
viewed = test_1[test_1['event_x_offer viewed']==1][['offer_id']]

In [111]:
offered = test_1[test_1['event_x_offer received']==1][['offer_id']]

In [164]:
transcript[(transcript['person']=='e2127556f4f64592b11af22de27a7932')&(transcript['event_x_offer viewed']==1)][['offer_id']]

,offer_id
17760,2906b810c7d4411798c6938adc9daaa5
152024,9b98b8c7a33c4b65b9aebfe6a799e6d9
199950,fafdcd668e3743c1bb461111dcafc2a4


In [119]:
offered.values

array([['2906b810c7d4411798c6938adc9daaa5'],
       ['3f207df678b143eea3cee63160fa8bed'],
       ['9b98b8c7a33c4b65b9aebfe6a799e6d9'],
       ['fafdcd668e3743c1bb461111dcafc2a4']], dtype=object)

In [140]:
offer_viewed = np.intersect1d(offered.values,viewed.values).tolist()

In [153]:
transcript.loc[(transcript['offer_id'].isin(offer_viewed))&(transcript['person'].isin(test_1_ids)),'purc_viewed']=True

In [156]:
transcript[transcript['person'].isin(test_1_ids)]

,index,time,person,offer_id,amount,reward,event_x_offer received,event_x_offer viewed,event_x_transaction,event_x_offer completed,purc_rewarded,purc_viewed
1,2,0,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,1,0,0,0,NaN,True
17760,20283,18,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,0,1,0,0,NaN,True
91402,102635,288,e2127556f4f64592b11af22de27a7932,NaN,17.88,NaN,0,0,1,0,NaN,False
92480,103834,294,e2127556f4f64592b11af22de27a7932,NaN,21.43,NaN,0,0,1,0,NaN,False
98717,110830,336,e2127556f4f64592b11af22de27a7932,3f207df678b143eea3cee63160fa8bed,NaN,NaN,1,0,0,0,NaN,False
133962,150600,408,e2127556f4f64592b11af22de27a7932,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,1,0,0,0,NaN,True
152024,171209,420,e2127556f4f64592b11af22de27a7932,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,0,1,0,0,NaN,True
179364,201574,504,e2127556f4f64592b11af22de27a7932,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,1,0,0,0,NaN,True
199950,225059,522,e2127556f4f64592b11af22de27a7932,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,0,1,0,0,NaN,True
199951,225060,522,e2127556f4f64592b11af22de27a7932,NaN,18.42,NaN,0,0,1,0,NaN,False


In [ ]:
def user_and_option(arr1, arr2):
    #function per user jterrace via Stack Overflow
    arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
    arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
    intersected = numpy.intersect1d(arr1_view, arr2_view)
    return intersected.view(arr1.dtype).reshape(-1, arr1.shape[1])